# Permutation analysis examining 4 classification models for simulated Bulk RNA-sequencing data

In [ ]:
import numpy as np
import pandas as pd
from time import time
import os,errno,dRFEtools
from sklearn.preprocessing import StandardScaler
from rpy2.robjects import r, pandas2ri, globalenv
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import normalized_mutual_info_score as nmi

## Functions

In [ ]:
def mkdir_p(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def load_data(simu):
    pandas2ri.activate()
    globalenv["simu"] = simu+1
    r('''
    suppressPackageStartupMessages(library(dplyr))
    counts <- data.table::fread(paste0("../../_m/bulk_data/simulated_counts_",simu,".tsv.gz")) %>%
        tibble::column_to_rownames("V1") %>% as.matrix
    phenotypes <- data.table::fread(paste0("../../_m/bulk_data/simulated_sampleInfo_", simu, ".tsv")) %>%
        tibble::column_to_rownames("V1")
    x <- edgeR::DGEList(counts=counts, samples=phenotypes)
    x <- edgeR::calcNormFactors(x, method="TMM")
    Z <- edgeR::cpm(x, log=TRUE) %>% as.data.frame
    ''')
    return r['Z'].T, r["phenotypes"]


def run_oob(estimator, x_train, x_test, y_train, y_test, fold, outdir, 
            frac, step, simu):
    features = x_train.columns
    d, pfirst = dRFEtools.rf_rfe(estimator, x_train, y_train, features, 
                                fold, outdir, elimination_rate=0.1, RANK=True)
    df_elim = pd.DataFrame([{'fold':fold, "simulation": simu,
                             'n features':k, 'NMI score':d[k][1], 
                             'Accuracy score':d[k][2], 
                             'ROC AUC score':d[k][3]} for k in d.keys()])
    n_features_max = max(d, key=lambda x: d[x][1])
    try:
        ## Max features from lowess curve
        n_features, _ = dRFEtools.extract_max_lowess(d, frac=frac, multi=False)
        n_redundant, _ = dRFEtools.extract_redundant_lowess(d, frac=frac, 
                                                            step_size=step, 
                                                            multi=False)
        dRFEtools.plot_with_lowess_vline(d, fold, outdir, frac=frac,
                                         step_size=step, multi=False)
    except ValueError:
        ## For errors in lowess estimate
        n_features = n_features_max 
        n_redundant = n_features
    ## Fit model
    estimator.fit(x_train, y_train)
    all_fts = estimator.predict(x_test)
    estimator.fit(x_train.iloc[:, d[n_redundant][4]], y_train)
    labels_pred_redundant = estimator.predict(x_test.iloc[:, d[n_redundant][4]])
    estimator.fit(x_train.iloc[:,d[n_features][4]], y_train)
    labels_pred = estimator.predict(x_test.iloc[:, d[n_features][4]])
    ## Output test predictions
    kwargs = {"average": "weighted"}
    pd.DataFrame({'fold': fold, "simulation": simu, 'real': y_test, 
                  'predict_all': all_fts, 'predict_max': labels_pred, 
                  'predict_redundant': labels_pred_redundant})\
      .to_csv("%s/test_predictions.txt" % outdir, sep='\t', mode='a', 
              index=True, header=True if fold == 0 else False)
    output = dict()
    output['fold'] = fold
    output['simulation'] = simu
    output['n_features'] = n_features
    output['n_redundant'] = n_redundant
    output['n_max'] = n_features_max
    output['train_nmi'] = dRFEtools.oob_score_nmi(estimator, y_train)
    output['train_acc'] = dRFEtools.oob_score_accuracy(estimator, y_train)
    output['train_roc'] = dRFEtools.oob_score_roc(estimator, y_train)
    output['test_nmi'] = nmi(y_test, labels_pred, average_method="arithmetic")
    output['test_acc'] = accuracy_score(y_test, labels_pred)
    output['test_roc'] = roc_auc_score(y_test, labels_pred, **kwargs)
    metrics_df = pd.DataFrame.from_records(output, index=[simu]).reset_index().drop('index', axis=1)
    return df_elim, metrics_df


def run_dev(estimator, x_train, x_test, y_train, y_test, fold, outdir, 
                 frac, step, simu):
    features = x_train.columns
    d, pfirst = dRFEtools.dev_rfe(estimator, x_train, y_train, features, 
                                 fold, outdir, elimination_rate=0.1, RANK=True)
    df_elim = pd.DataFrame([{'fold':fold, "simulation": simu,
                             'n features':k, 'NMI score':d[k][1], 
                             'Accuracy score':d[k][2], 
                             'ROC AUC score':d[k][3]} for k in d.keys()])
    n_features_max = max(d, key=lambda x: d[x][1])
    try:
        ## Max features from lowess curve
        ### multiple classification is False by default
        n_features, _ = dRFEtools.extract_max_lowess(d, frac=frac)
        n_redundant, _ = dRFEtools.extract_redundant_lowess(d, frac=frac, 
                                                            step_size=step)
        dRFEtools.plot_with_lowess_vline(d, fold, outdir, frac=frac, 
                                         step_size=step, multi=False)
    except ValueError:
        ## For errors in lowess estimate
        n_features = n_features_max 
        n_redundant = n_features
    ## Fit model
    #x_dev, x_test, y_dev, y_test = train_test_split(x_train, y_train)
    estimator.fit(x_train, y_train)
    all_fts = estimator.predict(x_test)
    estimator.fit(x_train.iloc[:, d[n_redundant][4]], y_train)
    labels_pred_redundant = estimator.predict(x_test.iloc[:, d[n_redundant][4]])
    estimator.fit(x_train.iloc[:,d[n_features][4]], y_train)
    labels_pred = estimator.predict(x_test.iloc[:, d[n_features][4]])
    ## Output test predictions
    kwargs = {"average": "weighted"}
    pd.DataFrame({'fold': fold, "simulation": simu, 'real': y_test, 
                  'predict_all': all_fts, 'predict_max': labels_pred, 
                  'predict_redundant': labels_pred_redundant})\
      .to_csv("%s/test_predictions.txt" % outdir, sep='\t', mode='a', index=True, 
              header=True if fold == 0 else False)
    output = dict()
    output['fold'] = fold
    output['simulation'] = simu
    output['n_features'] = n_features
    output['n_redundant'] = n_redundant
    output['n_max'] = n_features_max
    output['train_nmi'] = dRFEtools.dev_score_nmi(estimator, x_train.iloc[:,d[n_features][4]], y_train)
    output['train_acc'] = dRFEtools.dev_score_accuracy(estimator, x_train.iloc[:,d[n_features][4]], y_train)
    output['train_roc'] = dRFEtools.dev_score_roc(estimator, x_train.iloc[:,d[n_features][4]], y_train)
    output['test_nmi'] = nmi(y_test, labels_pred, average_method="arithmetic")
    output['test_acc'] = accuracy_score(y_test, labels_pred)
    output['test_roc'] = roc_auc_score(y_test, labels_pred, **kwargs)
    metrics_df = pd.DataFrame.from_records(output, index=[simu]).reset_index().drop('index', axis=1)
    return df_elim, metrics_df

## Generate 10-fold cross-validation

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=20210930)
scaler = StandardScaler()

### Logistic regression

#### Initial load

In [ ]:
outdir = "lr/"
mkdir_p(outdir)
cla = dRFEtools.LogisticRegression(n_jobs=-1, random_state=13, 
                                   max_iter=1000, penalty="l2")

#### Optimize

In [ ]:
X, Y = load_data(0)
y = Y.Group.astype("category").cat.codes

fold = 1
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    fold += 1
fold -= 1

features = X_train.columns
d, pfirst = dRFEtools.dev_rfe(cla, X_train, y_train, features, fold, 
                             outdir, elimination_rate=0.1, RANK=False)

#### Dynamic run

In [ ]:
cpu_lt = []; simu_lt = []
for simu in range(2):
    X, Y = load_data(simu)
    y = Y.Group.astype("category").cat.codes
    simu_out = "%s/simulate_%d" % (outdir, simu)
    mkdir_p(simu_out)
    ## default parameters
    frac = 0.3; step=0.05; fold = 0
    df_dict = pd.DataFrame(); output = pd.DataFrame()
    start = time()
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
        df_elim, metrics_df = run_dev(cla, X_train, X_test, y_train, y_test, fold, 
                                      simu_out, frac, step, simu)
        df_dict = pd.concat([df_dict, df_elim], axis=0)
        output = pd.concat([output, metrics_df], axis=0)
        fold += 1
    end = time()
    df_dict.to_csv("%s/dRFE_simulation_elimination.txt" % outdir,
                   sep='\t', mode='a', index=False, 
                   header=True if simu == 0 else False)
    output.to_csv("%s/dRFE_simulation_metrics.txt" % outdir,
                  sep='\t', mode='a', index=False, 
                  header=True if simu == 0 else False)
    cpu_lt.append(end - start)
    simu_lt.append(simu)
pd.DataFrame({"Simulation": simu_lt, "CPU Time": cpu_lt})\
  .to_csv("%s/simulation_time.csv" % outdir, index=False)

### SVC linear kernel

#### Initial load

In [ ]:
outdir = "svc/"
mkdir_p(outdir)
cla = dRFEtools.LinearSVC(random_state=13, max_iter=10000)

#### Optimize

In [ ]:
X, Y = load_data(0)
y = Y.Group.astype("category").cat.codes

fold = 1
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    fold += 1
fold -= 1

features = X_train.columns
d, pfirst = dRFEtools.dev_rfe(cla, X_train, y_train, features, fold, 
                             outdir, elimination_rate=0.1, RANK=False)

In [ ]:
for frac in [0.2, 0.25, 0.3, 0.35]:
    dRFEtools.optimize_lowess_plot(d, fold, outdir, frac=frac, step_size=0.05, 
                                   classify=True, save_plot=False)

#### Dynamic run

In [ ]:
cpu_lt = []; simu_lt = []
for simu in range(2):
    X, Y = load_data(simu)
    y = Y.Group.astype("category").cat.codes
    simu_out = "%s/simulate_%d" % (outdir, simu)
    mkdir_p(simu_out)
    ## default parameters
    frac = 0.3; step=0.05; fold = 0
    df_dict = pd.DataFrame(); output = pd.DataFrame()
    start = time()
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
        df_elim, metrics_df = run_dev(cla, X_train, X_test, y_train, y_test, fold, 
                                      simu_out, frac, step, simu)
        df_dict = pd.concat([df_dict, df_elim], axis=0)
        output = pd.concat([output, metrics_df], axis=0)
        fold += 1
    end = time()
    df_dict.to_csv("%s/dRFE_simulation_elimination.txt" % outdir,
                   sep='\t', mode='a', index=False, 
                   header=True if simu == 0 else False)
    output.to_csv("%s/dRFE_simulation_metrics.txt" % outdir,
                  sep='\t', mode='a', index=False, 
                  header=True if simu == 0 else False)
    cpu_lt.append(end - start)
    simu_lt.append(simu)
pd.DataFrame({"Simulation": simu_lt, "CPU Time": cpu_lt})\
  .to_csv("%s/simulation_time.csv" % outdir, index=False)

### SGD classifier

#### Initial load

In [ ]:
outdir = "sgd/"
mkdir_p(outdir)
cla = dRFEtools.SGDClassifier(random_state=13, loss="perceptron", n_jobs=-1)

#### Optimize

In [ ]:
X, Y = load_data(0)
y = Y.Group.astype("category").cat.codes

fold = 1
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    fold += 1
fold -= 1

features = X_train.columns
d, pfirst = dRFEtools.dev_rfe(cla, X_train, y_train, features, fold, 
                             outdir, elimination_rate=0.1, RANK=False)

In [ ]:
for frac in [0.2, 0.25, 0.3, 0.35]:
    dRFEtools.optimize_lowess_plot(d, fold, outdir, frac=frac, step_size=0.05, 
                                   classify=True, save_plot=False)

#### Dynamic run

In [ ]:
cpu_lt = []; simu_lt = []
for simu in range(2):
    X, Y = load_data(simu)
    y = Y.Group.astype("category").cat.codes
    simu_out = "%s/simulate_%d" % (outdir, simu)
    mkdir_p(simu_out)
    ## default parameters
    frac = 0.3; step=0.05; fold = 0
    df_dict = pd.DataFrame(); output = pd.DataFrame()
    start = time()
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]
        df_elim, metrics_df = run_dev(cla, X_train, X_test, y_train, y_test, fold, 
                                      simu_out, frac, step, simu)
        df_dict = pd.concat([df_dict, df_elim], axis=0)
        output = pd.concat([output, metrics_df], axis=0)
        fold += 1
    end = time()
    df_dict.to_csv("%s/dRFE_simulation_elimination.txt" % outdir,
                   sep='\t', mode='a', index=False, 
                   header=True if simu == 0 else False)
    output.to_csv("%s/dRFE_simulation_metrics.txt" % outdir,
                  sep='\t', mode='a', index=False, 
                  header=True if simu == 0 else False)
    cpu_lt.append(end - start)
    simu_lt.append(simu)
pd.DataFrame({"Simulation": simu_lt, "CPU Time": cpu_lt})\
  .to_csv("%s/simulation_time.csv" % outdir, index=False)

## Random forest classifier

In [ ]:
outdir = 'rf/'
mkdir_p(outdir)
cla = dRFEtools.RandomForestClassifier(n_estimators=100, oob_score=True, 
                                      n_jobs=-1, random_state=13)

### Optimize

In [ ]:
X, y = load_data(0)
fold = 1
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y[train_index], y[test_index]
    fold += 1
fold -= 1

features = X_train.columns

In [ ]:
d, pfirst = dRFEtools.rf_rfe(cla, X_train.values, y_train.values, features, 
                            fold, outdir, elimination_rate=0.2, RANK=False)

for frac in [0.2, 0.25, 0.3, 0.35]:
    dRFEtools.optimize_lowess_plot(d, fold, outdir, frac=frac, step_size=0.05, 
                                   classify=True, save_plot=False)

In [ ]:
for step_size in [0.01, 0.02, 0.03, 0.04, 0.05, 0.1]:
    dRFEtools.optimize_lowess_plot(d, fold, outdir, frac=0.3, step_size=step_size, 
                                   classify=True, save_plot=True)

## Classification example

In [ ]:
# make output directory
outdir = 'classification_simu/'
mkdir_p(outdir)

# Create a dataset with only 10 informative features

cla = RandomForestClassifier(n_estimators=100, oob_score=True, n_jobs=-1)

### RaFFE

#### Optimize regression

In [ ]:
print(raffe.extract_max_lowess(d, frac=0.35))
raffe.extract_redundant_lowess(d, frac=0.35)

#### Function to run RaFFE

In [ ]:
def run_raffe_cla(estimator, x_train, x_test, y_train, y_test, fold, outdir):
    features = ["feature_%d" % x for x in range(x_train.shape[1])]
    d, pfirst = raffe.feature_elimination(estimator, x_train, y_train, 
                                          np.array(features), 
                                          fold, outdir, 
                                          elimination_rate=0.1, 
                                          RANK=True)
    df_elim = pd.DataFrame([{'fold':fold,
                             'n features':k,
                             'normalized mutual information':d[k][1], 
                             'accuracy':d[k][2], 
                             'ROC AUC':d[k][3]} for k in d.keys()])
    n_features_max = max(d, key=lambda x: d[x][1])
    try:
        n_features,_ = raffe.extract_max_lowess(d, frac=0.35)
        n_redundant,_ = raffe.extract_redundant_lowess(d, frac=0.35)
        raffe.plot_with_lowess_vline(d, fold, outdir,
                                     classify=True)
        raffe.optimize_lowess_plot(d, fold, outdir, frac=0.35, step_size=0.05, 
                                   classify=True, save_plot=True)
    except ValueError:
        n_features = n_features_max 
    estimator.fit(x_train[:,d[n_features][4]], y_train)
    labels_pred = estimator.predict(x_test[:, d[n_features][4]])
    metrics_df = pd.DataFrame({'n_features_max': n_features_max, 
                               'n_features': n_features, 
                               'n_redundant': n_redundant,
                               'train_acc':raffe.oob_score_accuracy(estimator, y_train), 
                               'train_nmi':raffe.oob_score_nmi(estimator, y_train),
                               'train_roc':raffe.oob_score_roc(estimator, y_train), 
                               'test_acc':accuracy_score(y_test, labels_pred), 
                               'test_nmi':nmi(y_test, labels_pred,
                                              average_method='arithmetic'), 
                               'test_roc':roc_auc_score(y_test, labels_pred)}, 
                              index=[fold])
    return df_elim, metrics_df

In [ ]:
start = time()
df_dict = pd.DataFrame()
output = pd.DataFrame()
fold = 1
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    df_elim, metrics_df = run_raffe_cla(cla, X_train, X_test, y_train, 
                                        y_test, fold, outdir)
    df_dict = pd.concat([df_dict, df_elim], axis=0)
    output = pd.concat([output, metrics_df.reset_index()], axis=0)
    fold += 1
end = time()
print(f"Runtime of the program is {end - start}")

In [ ]:
li = output.set_index('index').loc[:, 'n_features'].mean()
lo = output.set_index('index').loc[:, 'n_features_max'].mean()
output.set_index('index').median()

In [ ]:
dft = pd.melt(df_dict, id_vars=['fold', 'n features'], 
              value_vars=['normalized mutual information', 'accuracy', 'ROC AUC'],
              var_name='Metrics', value_name='Score')

gg = ggplot(dft, aes(x='n features', y='Score', color='Metrics')) +\
    geom_jitter(size=1, alpha=0.6) + facet_wrap('~Metrics') +\
    geom_vline(xintercept=li, color='black', linetype='dashed') +\
    scale_x_log10() + theme_classic() + theme(legend_position="top")
save_plot(gg, '%s/raffe_feature_selection' % outdir, 12, 4)
gg